In [1]:
import numpy as np
import pandas as pd
import glob
import os
import requests
from docarray import DocumentArray
from docarray import dataclass
from docarray.typing import Image, Text
from docarray import Document

In [15]:
# explore unsplash dataset, if needed all csv files

path = './'
documents = ['photos', 'colors']
datasets = {}

for doc in documents:
    files = glob.glob(path + doc + ".tsv*")
    
    subsets = []
    for filename in files:
        df = pd.read_csv(filename, sep='\t', header=0)
        subsets.append(df)
    
    datasets[doc] = pd.concat(subsets, axis=0, ignore_index=True)

In [16]:
datasets['photos'].isna().sum()

photo_id                              0
photo_url                             0
photo_image_url                       0
photo_submitted_at                    0
photo_featured                        0
photo_width                           0
photo_height                          0
photo_aspect_ratio                    0
photo_description                 14098
photographer_username                 0
photographer_first_name               0
photographer_last_name             1582
exif_camera_make                   2812
exif_camera_model                  2852
exif_iso                           3192
exif_aperture_value                3600
exif_focal_length                  3503
exif_exposure_time                 3215
photo_location_name               15309
photo_location_latitude           17853
photo_location_longitude          17856
photo_location_country            16211
photo_location_city               18856
stats_views                           0
stats_downloads                       0


In [17]:
df = datasets['photos'].dropna(axis=0, subset=['ai_description'])

In [18]:
print(len(df))

23641


In [19]:
df.isna().sum()

photo_id                              0
photo_url                             0
photo_image_url                       0
photo_submitted_at                    0
photo_featured                        0
photo_width                           0
photo_height                          0
photo_aspect_ratio                    0
photo_description                 13325
photographer_username                 0
photographer_first_name               0
photographer_last_name             1472
exif_camera_make                   2631
exif_camera_model                  2668
exif_iso                           2984
exif_aperture_value                3371
exif_focal_length                  3281
exif_exposure_time                 3008
photo_location_name               14457
photo_location_latitude           16824
photo_location_longitude          16827
photo_location_country            15302
photo_location_city               17807
stats_views                           0
stats_downloads                       0


In [21]:
# get unique photo id's
photo_id_list = list(set(df['photo_id'].values))
print(len(photo_id_list))
# photo_id_list

23641


In [22]:
photo_image_url_list = list(set(df['photo_image_url'].values))
print(len(photo_image_url_list))
# photo_image_url_list

23641


In [24]:
photo_id_to_img_url_dict = {}

photo_id_to_img_url_dict = dict(zip(df.photo_id, df.photo_image_url))
print(len(photo_id_to_img_url_dict))
# photo_id_to_img_url_dict

23641


In [25]:
unsplash_lite_img_emb_da = DocumentArray()

for photo_id in photo_id_list:
    try:
        image_path = f'resize_images/{photo_id}.jpg'
        doc = Document(uri=image_path).load_uri_to_image_tensor()
        unsplash_lite_img_emb_da.append(doc)
    except:
        continue

In [26]:
unsplash_lite_img_emb_da.summary()

╭─────────────────────── Documents Summary ───────────────────────╮
│                                                                 │
│   Type                   DocumentArrayInMemory                  │
│   Length                 23599                                  │
│   Homogenous Documents   True                                   │
│   Common Attributes      ('id', 'tensor', 'mime_type', 'uri')   │
│   Multimodal dataclass   False                                  │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   id          ('str',)       23599            False             │
│   mime_type   ('str',)       1                False             │
│   tensor      ('ndarray',)   23599            False             │
│   uri         ('str',)       23599            False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

In [27]:
unsplash_lite_img_emb_da[0]

📄 Document: f7c16e033ebcfe832e6ee49300a2ccca
╭────────────┬─────────────────────────────────────────────────────────────────╮
│ Attribute  │ Value                                                           │
├────────────┼─────────────────────────────────────────────────────────────────┤
│ tensor     │ <class 'numpy.ndarray'> in shape (250, 250, 3), dtype: uint8    │
│ mime_type  │ image/jpeg                                                      │
│ uri        │ resize_images/A6pK6vCywBg.jpg                                   │
╰────────────┴─────────────────────────────────────────────────────────────────╯

### Embedding Creation by Model Inference  with ONNX

In [28]:
run_name = 'unsplash-lite-clip-run-onnx-11132022-2145'
artifact_name = 'unsplash-lite-clean-clip-onnx-model'

In [29]:
artifact_local_path = f"{artifact_name}/{run_name}.zip"

In [31]:
import finetuner

In [32]:
# Quick Test
image_da = DocumentArray([Document(uri='https://upload.wikimedia.org/wikipedia/commons/4/4e/Single_apple.png')])

clip_image_encoder = finetuner.get_model(artifact=artifact_local_path, select_model='clip-vision', is_onnx=True)

finetuner.encode(model=clip_image_encoder, data=image_da)

print(image_da.summary())
print(image_da.embeddings.shape)

Output()

╭──────────────────────── Documents Summary ─────────────────────────╮
│                                                                    │
│   Type                   DocumentArrayInMemory                     │
│   Length                 1                                         │
│   Homogenous Documents   True                                      │
│   Common Attributes      ('id', 'mime_type', 'uri', 'embedding')   │
│   Multimodal dataclass   False                                     │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   1                False             │
│   id          ('str',)       1                False             │
│   mime_type   ('str',)       1                False             │
│   uri         ('str',)       1                False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

None
(1, 512)


##### FillUp unsplash_lite_img_da with Embeddings created by our finetuned CLIP Image model

In [34]:
finetuner.encode(model=clip_image_encoder, data=unsplash_lite_img_emb_da)

Output()

In [35]:
unsplash_lite_img_emb_da.summary()

╭───────────────────────────── Documents Summary ──────────────────────────────╮
│                                                                              │
│   Type                   DocumentArrayInMemory                               │
│   Length                 23599                                               │
│   Homogenous Documents   True                                                │
│   Common Attributes      ('id', 'tensor', 'mime_type', 'uri', 'embedding')   │
│   Multimodal dataclass   False                                               │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   23599            False             │
│   id          ('str',)       23599            False             │
│   mime_type   ('str',)       1                False             │
│   tensor      ('ndarray',)   23599            False             │
│   uri         ('str',)       23599            False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

##### remove all tensors to decrease the size of embeddings

In [36]:
#remove all tensors to decrease the size of embeddings
del unsplash_lite_img_emb_da[:, 'tensor']

unsplash_lite_img_emb_da.summary()

╭──────────────────────── Documents Summary ─────────────────────────╮
│                                                                    │
│   Type                   DocumentArrayInMemory                     │
│   Length                 23599                                     │
│   Homogenous Documents   True                                      │
│   Common Attributes      ('id', 'mime_type', 'uri', 'embedding')   │
│   Multimodal dataclass   False                                     │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   23599            False             │
│   id          ('str',)       23599            False             │
│   mime_type   ('str',)       1                False             │
│   uri         ('str',)       23599            False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

#### Convert local URI to Online URI, so we can show images directly from Online 

In [37]:
for doc in unsplash_lite_img_emb_da:
    try:
#         print("original:", doc.uri)
        photo_id = str(str(doc.uri).split('/')[1]).split('.')[0]
#         print("photo_id:",photo_id)
#         print("photo_image_url:", photo_id_to_img_url_dict[photo_id])
        doc.uri = photo_id_to_img_url_dict[photo_id]
#         src_uri_unsplash_lite_emb_da.append(doc)
    except:
        print("couldn't convert=>", doc.uri)
        continue

In [38]:
unsplash_lite_img_emb_da.summary()

╭──────────────────────── Documents Summary ─────────────────────────╮
│                                                                    │
│   Type                   DocumentArrayInMemory                     │
│   Length                 23599                                     │
│   Homogenous Documents   True                                      │
│   Common Attributes      ('id', 'mime_type', 'uri', 'embedding')   │
│   Multimodal dataclass   False                                     │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   23599            False             │
│   id          ('str',)       23599            False             │
│   mime_type   ('str',)       2                False             │
│   uri         ('str',)       23599            False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

In [39]:
unsplash_lite_img_emb_da[0]

📄 Document: f7c16e033ebcfe832e6ee49300a2ccca
╭───────────┬──────────────────────────────────────────────────────────────────╮
│ Attribute │ Value                                                            │
├───────────┼──────────────────────────────────────────────────────────────────┤
│ mime_type │ image/jpeg                                                       │
│ uri       │ https://images.unsplash.com/photo-1566296906137-2367d1561746     │
│ embedding │ ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄ │
│           │ ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄ │
│           │ ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄ │
│           │ ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄ │
│           │ ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄ │
│           │ ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄[38;2;218;1

In [41]:
unsplash_lite_img_emb_da.save_binary("../frontend/unsplash_lite_img_emb_da.bin", compress='lz4')

In [ ]:
# data_da = DocumentArray.load_binary("../frontend/unsplash_lite_img_emb_da.bin", compress='lz4')